#Loading dataset

In [ ]:
!pip install datasets
!pip install tensorflow
!pip install livelossplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset

#dataset = load_dataset('conll2003', use_auth_token=True, streaming= True)

raw_datasets = load_dataset('conll2003')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

NER TAGS: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


In [ ]:
column_names = raw_datasets["train"].column_names
features = raw_datasets["train"].features

In [ ]:
column_names

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

In [ ]:
text_column_name = column_names[1]
text_column_name

'tokens'

In [ ]:
label_column_name = column_names[4]
label_column_name

'ner_tags'

In [ ]:
def get_label_list(labels):
      unique_labels = set()
      for label in labels:
          unique_labels = unique_labels | set(label)
      label_list = list(unique_labels)
      label_list.sort()
      return label_list

In [ ]:
features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [ ]:
label_list = features[label_column_name].feature.names

In [ ]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
num_labels = len(label_list)
num_labels

9

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification, AutoModelForTokenClassification, get_cosine_with_hard_restarts_schedule_with_warmup, AdamW
import torch
from torch.optim import AdamW
import numpy as np

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

#Tokenizer and data split

In [ ]:
from transformers import DataCollatorForTokenClassification

model_checkpoint = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
train_dataset = raw_datasets['train']
train_dataset  = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    desc="Running tokenizer on train dataset"
)
train_dataset

In [ ]:
eval_dataset = raw_datasets['validation']
eval_dataset = eval_dataset.map(
                tokenize_and_align_labels,
                batched=True,
                desc="Running tokenizer on validation dataset",
            )
eval_dataset

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
!pip install seqeval
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Metric calculation function

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        # Remove ignored index (special tokens)
        true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        results = metric.compute(predictions=true_predictions, references=true_labels)
  
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

#Creating model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    )


layers = model.bert.encoder.layer[:3]
for layer in layers:
    for param in layer.parameters():
        param.requires_grad = False

device = torch.device('cuda') if torch.cuda.is_available() \
        else torch.device('cpu')

model.to(device=device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

#Initialize wandb

In [ ]:
!pip install wandb

In [ ]:
import wandb

In [ ]:
wandb.init(project="SI-project-bert")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vshishunjy. Use `wandb login --relogin` to force relogin


#Training

In [ ]:
num_of_epochs = 4
batch_size_train = 64
batch_size_eval = 64
train_lr = 5e-5
weight_decay=0.01

In [ ]:
args = TrainingArguments(
    model_checkpoint,
    per_device_train_batch_size = batch_size_train,
    per_device_eval_batch_size = batch_size_eval,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=train_lr,
    num_train_epochs=num_of_epochs,
    weight_decay=weight_decay,
    report_to="wandb"
)


In [ ]:
from transformers import get_cosine_schedule_with_warmup
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
print("TRAINING")
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos_tags, id, ner_tags, chunk_tags. If tokens, pos_tags, id, ner_tags, chunk_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 4
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 880
  Number of trainable parameters = 87634953
Automatic Weights & Biases logging enabled, to disable set os.environ["

TRAINING


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.056035,0.908927,0.930495,0.919584,0.984813
2,No log,0.048228,0.926337,0.941770,0.933990,0.986449
3,0.105100,0.052065,0.925467,0.942444,0.933878,0.986052
4,0.105100,0.051549,0.929897,0.944295,0.937041,0.986624


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos_tags, id, ner_tags, chunk_tags. If tokens, pos_tags, id, ner_tags, chunk_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 64
Saving model checkpoint to bert-base-uncased/checkpoint-220
Configuration saved in bert-base-uncased/checkpoint-220/config.json
Model weights saved in bert-base-uncased/checkpoint-220/pytorch_model.bin
tokenizer config file saved in bert-base-uncased/checkpoint-220/tokenizer_config.json
Special tokens file saved in bert-base-uncased/checkpoint-220/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos_tags, id, ner_tags, chunk_tags. If tokens, pos_tags, id, ner_tags, chunk

TrainOutput(global_step=880, training_loss=0.06871476877819407, metrics={'train_runtime': 541.1404, 'train_samples_per_second': 103.788, 'train_steps_per_second': 1.626, 'total_flos': 1650948806003760.0, 'train_loss': 0.06871476877819407, 'epoch': 4.0})

#Evaluation

In [ ]:
print("EVALUATION")
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, pos_tags, id, ner_tags, chunk_tags. If tokens, pos_tags, id, ner_tags, chunk_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 64


EVALUATION


{'eval_loss': 0.05154871568083763,
 'eval_precision': 0.9298972489227709,
 'eval_recall': 0.9442948502187816,
 'eval_f1': 0.9370407481629925,
 'eval_accuracy': 0.9866236675298267,
 'eval_runtime': 12.547,
 'eval_samples_per_second': 259.027,
 'eval_steps_per_second': 4.065,
 'epoch': 4.0}

In [ ]:
trainer.save_model('bert_model_save')

In [ ]:
from google.colab import files
files.download('bert-base-uncased') 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')